In [5]:
from arcgis.gis import GIS
import os
import datetime

In [6]:
###USER DEFINED PARAMETERS

# Login parameters
output_location = r'C:\temp'  # Location where outputs will be saved
USERNAME = ''  # Your DIEM username
PASS = ''  # Your password

# Survey parameters
iso3 = ''  # ISO3 3-letters code of the country - leave empty for all countries. 
round = ''  # DIEM survey round number (integer - leave empty for all rounds)

# Download parameters
return_geometry_in_aggregated_data = False  # Include geometry in aggregated data
geometry_export_format = 'File Geodatabase'  # Export format: Shapefile, CSV, File Geodatabase, etc.

In [7]:
# Get the current date and time
current_datetime = datetime.datetime.now()
# Format the date and time as YYYYMMDDhhmmss
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

# Initialize the where clause
where_clause = ""

# Build the where clause based on conditions
if iso3 and round:
    where_clause = f"adm0_iso3 = '{iso3}' and round = {round}"
elif iso3:
    where_clause = f"adm0_iso3 = '{iso3}'"
elif round:
    where_clause = f"round = {round}"
else:
    where_clause = "1=1"  # Select all records if no filters are set

In [9]:
# Connect to the GIS
gis = GIS('https://hqfao-hub.maps.arcgis.com', USERNAME, PASS)

# List of predefined item IDs
item_ids = [
    '1b006938d6a344aeb5a309f69f3e344b',
    'fbef5b1ef85840838166a6b4d359f9bb',
    '499917f1518141209c2a6de55a79d991',
    '71460258c059453d8eab2d7c56a7b0c5'
]

for item_id in item_ids:
    group_item = gis.content.get(item_id)
    print(f"Processing the {group_item.title} dataset.")
    if return_geometry_in_aggregated_data:
        subset_feature_layer_id = group_item.layers[0].properties.id
        parameters = {"layers": [{"id": subset_feature_layer_id, "where": where_clause}]}
        # Export the data with geometry included
        result = group_item.export(
            f'{group_item.title}_{formatted_datetime}', geometry_export_format, parameters=parameters
        )
        result.download(output_location)
        result.delete()  # Clean up by deleting the export from AGOL
        print(f"Saved the {group_item.title} data in {output_location}.\n")
    else:
        subset_feature_layer = group_item.layers[0]
        df = subset_feature_layer.query(
            where=where_clause, out_fields="*", returnGeometry=False, as_df=True
        )
        df = df.drop(columns=['SHAPE'], errors='ignore')  # Exclude the SHAPE field if it exists
        df.to_csv(os.path.join(output_location, f"{group_item.title}_{formatted_datetime}.csv"))
        print(f"Saved the {group_item.title} data in {output_location}.\n")

print("Process completed.")

Processing the DIEM aggregated data (crop production thematic area) dataset.
Saved the DIEM aggregated data (crop production thematic area) data in C:\temp.

Processing the DIEM aggregated data (food security thematic area) dataset.
Saved the DIEM aggregated data (food security thematic area) data in C:\temp.

Processing the DIEM aggregated data (income, shocks and needs thematic areas) dataset.
Saved the DIEM aggregated data (income, shocks and needs thematic areas) data in C:\temp.

Processing the DIEM aggregated data (livestock production thematic area) dataset.
Saved the DIEM aggregated data (livestock production thematic area) data in C:\temp.

Process completed.
